# 현재 적용할 수 있는 평가 내용
- EB : 빈 마디 비율 : 
- UPC : pitch class의 비율
- QN : 3 timestep 보다 짧은 노트들의 비율
- DP : 드럼이 없으므로 측정하지 않음
- TD : 판별 조건이 애매하므로 측정하지 않음
  - 마디 간 화성의 유사성이 의미가 없을 듯
- CTR : 음이 코드 톤과 일치하는 비율

In [2]:
import pickle

with open('./MuseGAN/melodyMuseGAN/data/melody_train_data.pkl', "rb") as f:
  melody = pickle.load(f)

with open('./MuseGAN/chordRNN/data/data_train.pkl', "rb") as f:
  chord = pickle.load(f)

In [3]:
import evaluator

In [3]:
print(melody.shape)
print(chord.shape)

(10143, 128, 40)
(10143, 128, 12)


In [4]:
sum_eb = 0
sum_upc = 0
sum_qn = 0
sum_ctr = 0
for i in range(melody.shape[0]):
    e = evaluator.Evaluator(melody[i], chord[i])
    spec = e.evaluate()
    sum_eb += spec['Empty Bars']
    sum_upc += spec['Used Pitch Class']
    sum_qn += spec['Qualified Notes']
    sum_ctr += spec['Code Tone Ratio']

print('avg_eb:', sum_eb/melody.shape[0])
print('avg_upc:', sum_upc/melody.shape[0])
print('avg_qn:', sum_qn/melody.shape[0])
print('avg_ctr:', sum_ctr/melody.shape[0])

avg_eb: 0.0034260080843931777
avg_upc: 2.0206299911268855
avg_qn: 0.6509368780279224
avg_ctr: 0.5029597222662033


In [5]:
import numpy as np
import torch
#m = np.load('MuseGAN_integrated/melodyMuseGAN/results/melody/score_hold_enforce_g_670.npy')
m = np.load('MuseGAN/melodyMuseGAN/result/generated_score.npy')
#m = np.load('MuseGAN_integrated/melodyMuseGAN/results/melody/score_G_FC_D_LSTM_g_700.npy')
m = m.reshape(128, 52)
p = m[:, :37]
o = m[:, 37:40]
ch = m[:, 40:]

p = torch.tensor(p)
o = torch.tensor(o)
ch = torch.tensor(ch)

p = torch.argmax(p, dim=-1)
o = torch.argmax(o, dim=-1)
ch = torch.argmax(ch, dim=-1)

p = torch.nn.functional.one_hot(p, num_classes=37)
o = torch.nn.functional.one_hot(o, num_classes=3)
m = torch.concat([p, o], dim=-1)
ch = torch.nn.functional.one_hot(ch, num_classes=12)

m = m.numpy()
ch = ch.numpy()

e = evaluator.Evaluator(m, ch)
e.evaluate()

{'Empty Bars': 0.0,
 'Used Pitch Class': 4.375,
 'Qualified Notes': 0.45714285714285713,
 'Code Tone Ratio': 0.15625}

In [5]:
e.show_df()

,Empty Bars,Used Pitch Class,Qualified Notes,Code Tone Ratio
0,0.0,6.75,0.515152,0.148438
